In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### pip install
https://coding-potato.tistory.com/7

In [ ]:
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user
# 설치 후 런타임 다시시작

In [1]:
import sklearn
sklearn.__version__

'0.23.2'

In [2]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 4.9 MB/s 
     |████████████████████████████████| 2.0 MB 43.3 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 10.4 MB 5.5 MB/s 
     |████████████████████████████████| 88 kB 7.8 MB/s 
     |████████████████████████████████| 262 kB 67.3 MB/s 
     |████████████████████████████████| 1.3 MB 28.2 MB/s 
     |████████████████████████████████| 17.9 MB 47.3 MB/s 
     |████████████████████████████████| 167 kB 51.5 MB/s 
     |████████████████████████████████| 134 kB 56.0 MB/s 
     |████████████████████████████████| 1.7 MB 43.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 

In [3]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [ ]:
!pip install markupsafe==2.0.1
# 설치 후 런타임 다시시작

In [1]:
import jinja2
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
pip install optuna

In [3]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [ ]:
pip install catboost

#### import

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import joblib
import os
import time
import glob
import itertools
from itertools import permutations, combinations

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from itertools import permutations, combinations
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor, plot_tree, plot_importance
from lightgbm import LGBMRegressor

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostRegressor,  HistGradientBoostingRegressor, StackingRegressor, RandomForestRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning) #경고메세지 끄기
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def RMSE(y, y_pred):
    return mean_squared_error(y, y_pred)**0.5

trainset = pd.read_csv('./drive/MyDrive/input/쇼핑몰 지점별 매출액 예측/train.csv')
testset = pd.read_csv('./drive/MyDrive/input/쇼핑몰 지점별 매출액 예측/test.csv')
sample_submission = pd.read_csv("./drive/MyDrive/input/쇼핑몰 지점별 매출액 예측/sample_submission.csv")

In [6]:
# n주차 만들기
from datetime import timedelta, datetime
def change_df(df):
    df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
    df['Date'] = df['Date'].astype('str')
    return df

trainset = change_df(trainset)
testset = change_df(testset)

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day=1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days=6-firstday.weekday())
    return (target - origin).days // 7 + 1

def week(df):
    a = []
    for i in df['Date'].values:
        yi = int(i[:4])
        mi = int(i[5:7])
        di = int(i[8:])
        a.append(get_week_no(yi, mi, di))

    df['week'] = a
    return df

def change_week(df):
    df = week(df)
    df['Date1'] = df['Date'].str[:7]
    t = df[df['week']==0]
    t = t.drop_duplicates(['Date1'])
    df.loc[df["Date1"].isin(t["Date1"]), "week"] = df.loc[df["Date1"].isin(t["Date1"]), "week"] + 1
    return df

def date_conv(df):
    df = change_week(df)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Week'] = df['Date'].dt.isocalendar().week.apply(lambda x: int(x))
    df['Year'] = df['Date'].dt.year.apply(lambda x: int(x))
    df['Month'] =df['Date'].dt.month.apply(lambda x: int(x))
    df['Day'] =df['Date'].dt.day.apply(lambda x: int(x))
    df['month_day'] =df['Date'].dt.strftime('%m-%d')
    #df['Weekday'] = df['Date'].dt.weekday
    return df
    
trainset = date_conv(trainset)
testset = date_conv(testset)

# date 삭제
trainset = trainset.drop(columns=['Date', 'month_day', 'Date1'])
testset = testset.drop(columns=['Date', 'month_day', 'Date1'])
trainset.shape

(6255, 17)

In [7]:
# 결측치, 스케일링
def holiday_to_number(isholiday):
    if isholiday == True:
        number = 1
    else:
        number = 0
    return number

def preprocessing(df):
    df['IsHoliday'] = df['IsHoliday'].apply(holiday_to_number)
    return df

train_copy = trainset.copy()
test_copy = testset.copy()

def null(df): 
    df = preprocessing(df)
    df = IterativeImputer(random_state=2021).fit_transform(df)
    df = pd.DataFrame(df)
    if df.shape[1] == train_copy.shape[1]:
        df.columns = train_copy.columns
    else:
        df.columns = test_copy.columns 
    return df

def scaler(df):
    df = null(df)
    scaled = StandardScaler().fit_transform(df[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']])
    df[['Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5']] = scaled
    df = df.drop(columns=['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5'])
    return df

# 정수형
def c_int(df):
    df = scaler(df)
    df['Store'] = df['Store'].apply(lambda x: int(x))
    df['Year'] = df['Year'].apply(lambda x: int(x))
    df['Month'] =df['Month'].apply(lambda x: int(x))
    df['Day'] =df['Day'].apply(lambda x: int(x))
    df['Week'] =df['Week'].apply(lambda x: int(x))
    df['week'] =df['week'].apply(lambda x: int(x))
    return df
    
trainset = c_int(trainset)
testset = c_int(testset)
trainset

,id,Store,Temperature,Fuel_Price,Unemployment,IsHoliday,Weekly_Sales,week,Week,Year,Month,Day,Scaled_Promotion1,Scaled_Promotion2,Scaled_Promotion3,Scaled_Promotion4,Scaled_Promotion5
0,1.0,1,42.31,2.572,8.106,0.0,1643690.90,1,5,2010,2,5,0.600356,0.197531,0.311259,0.498490,0.464308
1,2.0,1,38.51,2.548,8.106,1.0,1641957.44,2,6,2010,2,12,0.598899,0.196968,0.310245,0.497338,0.462904
2,3.0,1,39.93,2.514,8.106,0.0,1611968.17,3,7,2010,2,19,0.579246,0.187278,0.291279,0.481808,0.438694
3,4.0,1,46.63,2.561,8.106,0.0,1409727.59,4,8,2010,2,26,0.448549,0.121946,0.162965,0.379030,0.275479
4,5.0,1,46.50,2.625,8.106,0.0,1554806.68,1,9,2010,3,5,0.542664,0.168812,0.254942,0.453044,0.392594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45,75.09,3.867,8.684,0.0,734297.87,5,35,2012,8,31,3.195838,-0.652682,-0.213507,1.128857,-0.120385
6251,6252.0,45,75.70,3.911,8.684,1.0,766512.66,1,36,2012,9,7,0.761870,-0.651263,-0.220785,-0.248151,-0.633371
6252,6253.0,45,67.87,3.948,8.684,0.0,702238.27,2,37,2012,9,14,0.835852,-0.118019,-0.229512,0.172164,0.217853
6253,6254.0,45,65.32,4.038,8.684,0.0,723086.20,3,38,2012,9,21,0.265646,-0.634680,-0.218869,-0.108235,1.118949


#### 변수생성

In [ ]:
from itertools import permutations, combinations
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

target_columns = ['Month', 'Year', 'Week']

combis = list(combinations(target_columns, 2))

for com in combis:
    trainset['{}*{}'.format(com[0], com[1])] = trainset['{}'.format(com[0])] * trainset['{}'.format(com[1])]
    testset['{}*{}'.format(com[0], com[1])] = testset['{}'.format(com[0])] * testset['{}'.format(com[1])]
    trainset['{}+{}'.format(com[0], com[1])] = trainset['{}'.format(com[0])] + trainset['{}'.format(com[1])]
    testset['{}+{}'.format(com[0], com[1])] = testset['{}'.format(com[0])] + testset['{}'.format(com[1])]
    trainset['{}/{}'.format(com[0], com[1])] = trainset['{}'.format(com[0])] / trainset['{}'.format(com[1])]
    testset['{}/{}'.format(com[0], com[1])] = testset['{}'.format(com[0])] / testset['{}'.format(com[1])]
    
print(trainset.shape)
# 생성후 결측치 확인
print(trainset.isnull().sum().sort_values())

(6255, 26)
id                   0
Year*Week            0
Month/Week           0
Month+Week           0
Month*Week           0
Month/Year           0
Month+Year           0
Month*Year           0
Scaled_Promotion5    0
Scaled_Promotion4    0
Scaled_Promotion3    0
Scaled_Promotion2    0
Scaled_Promotion1    0
Day                  0
Month                0
Year                 0
Week                 0
week                 0
Weekly_Sales         0
IsHoliday            0
Unemployment         0
Fuel_Price           0
Temperature          0
Store                0
Year+Week            0
Year/Week            0
dtype: int64


#### pycaret 쓰기

In [9]:
from catboost import CatBoostClassifier, CatBoostRegressor
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning) #경고메세지 끄기
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
# 모델링
cat = CatBoostRegressor()
features = ['Day', 'Month', 'Year', 'IsHoliday'] 
models = []
error = []
from tqdm import tqdm
for store in tqdm(range(11,21)):
    train_store = trainset[trainset.Store==store]
    
    # 연도별 데이터 분해
    train2010 = train_store[(train_store.Year==2010) & (train_store.Month<=9)]
    train2011 = train_store[(train_store.Year==2011) & (train_store.Month<=9)]
    train2012 = train_store[(train_store.Year==2012) & (train_store.Month<9)]
    
    # 3가지(2010+2012, 2011+2012, 2010+2011+2012) 데이터프레임 생성
    train0 = pd.concat([train2010, train2012])
    train1 = pd.concat([train2011, train2012])
    train2 = pd.concat([train2010, train2011, train2012])
    
    # 각각의 모델 학습
    model0 = cat
    model0.fit(train0[features], train0.Weekly_Sales)
    
    model1 = cat
    model1.fit(train1[features], train1.Weekly_Sales)
    
    model2 = cat
    model2.fit(train2[features], train2.Weekly_Sales)
    
    # 2012년도 9월에 대해서 예측
    y_val = train_store[(train_store.Year==2012) & (train_store.Month==9)].Weekly_Sales
    X_val = train_store[(train_store.Year==2012) & (train_store.Month==9)][features]

    pred0 = model0.predict(X_val)
    pred1 = model1.predict(X_val)
    pred2 = model2.predict(X_val)
    
    # RMSE
    rmse0 = RMSE(pred0, y_val)
    rmse1 = RMSE(pred1, y_val)
    rmse2 = RMSE(pred2, y_val)
    
    # 최종 데이터 구축
    if rmse0 < rmse1 and rmse0 < rmse2:
        error.append(rmse0)
        train_s = train_store[(train_store.Year==2010) & (train_store.Month<=10)]
    elif rmse1 < rmse2: 
        error.append(rmse1)
        train_s = train_store[(train_store.Year==2011) & (train_store.Month<=10)]
    else:
        error.append(rmse2)
        train_s = pd.concat([train_store[(train_store.Year==2010) & (train_store.Month<=10)],
                             train_store[(train_store.Year==2011) & (train_store.Month<=10)]])
          
    # 최종 데이터 구축
    train_final = pd.concat([train_s, train_store[train_store.Year==2012]])
    X_train = train_final[features]
    y_train = train_final['Weekly_Sales']

    # 모델링
    model = setup(train_final, target = 'Weekly_Sales', fold_shuffle=True, fold=10, 
                  session_id=530, remove_multicollinearity = True,ignore_low_variance = True,
                  pca = False,normalize= True, n_jobs = -1, silent = True, remove_outliers= True
                  )
    top5 = compare_models(exclude = ['svm'],round=4,n_select=5,sort='RMSE')
    tuned_top5 = [tune_model(i) for i in top5] # 모델 안의 파라미터 값 튜닝
    blender_top5 = blend_models(estimator_list=tuned_top5) # 5개의 모델을 혼합한 모델
    final_model = finalize_model(blender_top5)

    # 모델 학습
    final_model = final_model.fit(X_train, y_train)
    models.append(final_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,56225.5534,3.848218e+09,62034.0103,0.8766,0.0327,0.0298
1,32845.6106,1.522873e+09,39024.0001,0.7379,0.0213,0.0179
2,36637.2493,2.866320e+09,53538.0249,0.8371,0.0276,0.0189
3,75567.2483,1.209957e+10,109998.0389,0.6111,0.0574,0.0401
4,33576.3002,1.608357e+09,40104.3313,0.9319,0.0213,0.0181
5,31736.4781,1.508013e+09,38833.1391,0.8687,0.0202,0.0167
6,72580.7244,1.798534e+10,134109.4458,0.2072,0.0666,0.0354
7,39235.5801,5.031902e+09,70935.9039,0.7369,0.0408,0.0229
8,43969.5730,3.540955e+09,59505.9265,0.4409,0.0315,0.0226


100%|██████████| 10/10 [30:01<00:00, 180.17s/it]


In [10]:
# 모델 저장
import joblib

for i in range(20):
  joblib.dump(models[i], open('model{0}.model'.format(i), 'wb'))

IndexError: ignored

In [62]:
# 모델 불러오기
for i in range(46):
  globals()['model{0}'.format(i)] = joblib.load(open('model{0}.model'.format(i), 'rb'))
  globals()['model{0}'.format(i)] 

# 모델 예측
preds = []
for i in range(46):
  test = testset[testset.Store==(i+1)]
  X_test = test[['Day', 'Month', 'Year', 'IsHoliday']]
  pred = globals()['model{0}'.format(i)].predict(X_test)
  preds.append(pred)

pred = list(itertools.chain.from_iterable(preds))

# 제출
sample_submission["Weekly_Sales"] = pred
sample_submission.to_csv('pycaret_model.csv',index = False)
sample_submission